# Transformations with Pandas

Sometimes if we have a question about our dataset, we may not be able to answer it directly, even if we have all of the data needed. It may happen that we will need a few additional steps in between where we will transform our data to a representation that will be easier for later data analysis. 

Today, we will learn how to modify our dataframe - transform the information we already have, add new columns, make some calculations... :)

We will also cover KPIs topic.

![data_tasks.jpg](https://thumbor.forbes.com/thumbor/960x0/https%3A%2F%2Fblogs-images.forbes.com%2Fgilpress%2Ffiles%2F2016%2F03%2FTime-1200x511.jpg)

## Let's remember cars dataset:
The dataset contains the information of 10.000 cars. There are 9 different columns:
- Make (Car brand, example: Ford)
- Model (The Model of the Car, example: Focus)
- Year (The Year in which the car was build, example: 2012)
- Variant (The car model version showing the PS, example: 1.6 Trendline)
- Kms (The kilometers the cars has been driven, example: 90000)
- Price (The offered price for the car, example: 10000)
- Doors (How many doors the car has, example: 4)
- Kind (Type of car, example: Pick-Up)
- Location (Where the car is located, example: Buenos Aires)


## Prepare the dataset

In [ ]:
# Imports
import pandas as pd
import plotly

In [ ]:
# install plotly
# https://plotly.com/python/pandas-backend/
!pip install plotly==4.14.3
pd.options.plotting.backend = "plotly"

In [ ]:
# read in the data
cars = pd.read_csv("https://raw.githubusercontent.com/juliandnl/redi_ss20/master/cars.csv")


In [ ]:
cars.sample(5)

,Make,Model,Year,Variant,Kms,Price,Doors,Kind,Location
5219,Volkswagen,Suran,2011,1.6 Trendline 11b,89000,180000,5.0,Monovolumen,Bs.as. G.b.a. Norte
8932,Volkswagen,Vento,2011,2.0 Sportline Tsi 200cv,109000,330000,4.0,Sedán,Buenos Aires Interior
8249,Volkswagen,Vento,2013,2.5 Luxury 170cv Tiptronic,77000,389500,4.0,Sedán,Bs.as. G.b.a. Oeste
2007,Volkswagen,Suran,2013,1.6 Imotion Highline,70000,245000,5.0,Monovolumen,Bs.as. G.b.a. Norte
1602,Volkswagen,Fox,2013,1.6 Confortline,55000,215000,5.0,Hatchback,Bs.as. G.b.a. Norte


## Let's answer some questions!

1. How old (in years) are the cars in our dataset?
2. What is the min and max kilometers per year that the cars in the dataset traveled?
3. What is the distribution of horsepower?

### 1. How old (in years) are the cars in our dataset?

We don't have a column providing us a direct information to answer this question. However, we have `Year` column which we can use to extract the information from.

Let's do it step by step!

What date do we have right now?

We can use a hardcoded value for that like this:

In [ ]:
today_hardcoded = 2021

But we can also get this value programatically, so we can run the code whenever we want and we don't need to care about checking if all the used values are up to date. 

In [ ]:
from datetime import date

today = date.today()
today

datetime.date(2021, 3, 10)

In [ ]:
today.year

2021

Now, when we already have the information about the current day, let's try to calculate the age of the first car in our dataset:

In [ ]:
first_car = cars.loc[0]
# You can also use iloc function, like this:
# first_car = cars.iloc[0]
# Here, it doesn't matter which function you will use, however there are differences between those two. 
# You can read more about this here: https://www.analyticsvidhya.com/blog/2020/02/loc-iloc-pandas/
first_car

Make              Volkswagen
Model                  Vento
Year                    2012
Variant     2.5 Luxury 170cv
Kms                    99950
Price                 360000
Doors                      4
Kind                   Sedán
Location             Córdoba
Name: 0, dtype: object

In [ ]:
car_age_in_years = today.year - first_car.Year
car_age_in_years

9

Alright! Right now we know how to calculate a single age difference. Let's see how to automate this process and calculate this information for all of the cars at once.

## `Apply` and `lambda`

In order to do that we will use `apply` function with `lambda` function: 

In [ ]:
today = date.today()

cars['Age'] = cars.Year.apply(lambda x: today.year - x)

In [ ]:
cars.head()

,Make,Model,Year,Variant,Kms,Price,Doors,Kind,Location,Age
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,4.0,Sedán,Córdoba,9
1,Ford,Ranger,2012,2.3 Cd Xl Plus 4x2,140000,320000,2.0,Pick-Up,Entre Ríos,9
2,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,5.0,Hatchback,Bs.as. G.b.a. Sur,10
3,Ford,Ranger,2017,3.2 Cd Xls Tdci 200cv Automática,13000,798000,4.0,Pick-Up,Neuquén,4
4,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,3.0,Hatchback,Córdoba,8


A `lambda` function is a small anonymous function. A lambda function can take any number of arguments, but can only have one expression.

## Python functions

This syntax will work great if our function is not very complex. However, there is also an option if we want to perform more complicated calculation - we just need to create a python function!

In [ ]:
today = date.today()

# A function calculating the age of a car.
def calculate_age_of_a_car(year):
    return today.year - year

cars['Age'] = cars.Year.apply(calculate_age_of_a_car)

In [ ]:
cars.head()

,Make,Model,Year,Variant,Kms,Price,Doors,Kind,Location,Age
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,4.0,Sedán,Córdoba,9
1,Ford,Ranger,2012,2.3 Cd Xl Plus 4x2,140000,320000,2.0,Pick-Up,Entre Ríos,9
2,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,5.0,Hatchback,Bs.as. G.b.a. Sur,10
3,Ford,Ranger,2017,3.2 Cd Xls Tdci 200cv Automática,13000,798000,4.0,Pick-Up,Neuquén,4
4,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,3.0,Hatchback,Córdoba,8


Let's answer our question: How old (in years) are the cars in our dataset?

In [ ]:
ages = cars.Age.value_counts()
ages.plot.bar()

### 2. What is the min and max kilometers per year that the cars in the dataset traveled?

The procedure here will be very similar. We will create a new column `KM_per_year` to answer this question.

In [ ]:
# A function calculating kilometers per year.
def calculate_km_per_year(row):
    return row.Kms / row.Age

cars['KM_per_year'] = cars.apply(calculate_km_per_year, axis=1)

Notice, that we have used an additional argument with the apply function: `axis`. We need that here since we will use more than one column to calculate a new value. In order to do that we will pass the whole row. And the `axis` parameter is just telling us in which direction we want to read our dataframe (Pandas Dataframe is a 2D structure which could be read as rows or as columns). 

In [ ]:
cars.head()

,Make,Model,Year,Variant,Kms,Price,Doors,Kind,Location,Age,KM_per_year,Horsepower,Sold_date
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,4.0,Sedán,Córdoba,9,11105.555556,2.5,2019-08-18 12:00:00
1,Ford,Ranger,2012,2.3 Cd Xl Plus 4x2,140000,320000,2.0,Pick-Up,Entre Ríos,9,15555.555556,2.3,2019-03-22 12:00:00
2,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,5.0,Hatchback,Bs.as. G.b.a. Sur,10,13200.000000,1.6,2019-05-11 12:00:00
3,Ford,Ranger,2017,3.2 Cd Xls Tdci 200cv Automática,13000,798000,4.0,Pick-Up,Neuquén,4,3250.000000,3.2,2019-03-17 12:00:00
4,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,3.0,Hatchback,Córdoba,8,13375.000000,1.4,2019-07-13 12:00:00


In [ ]:
round(cars.KM_per_year.max(), 1)

22857.1

In [ ]:
cars.KM_per_year.min()

680.0

The maximum `kilometer per year` is 22857.1 km/year and the minimum is 680 km/year.

### 3. What is the distribution of horsepower?

How to find horsepower? If you take a look on `Variant` column you will see that it contains a few information there. But the format is always the same - it starts with a number representing the horsepower. And we can extract this information from there.

In [ ]:
# A function extracting horsepower from `Variant` column
def get_horsepower(variant):
    # Extract the first element from our string. (More on strings next week!)
    return variant.split(' ')[0]

cars['Horsepower'] = cars.Variant.apply(lambda x: get_horsepower(x))

In [ ]:
cars.head()

,Make,Model,Year,Variant,Kms,Price,Doors,Kind,Location,Age,KM_per_year,Horsepower
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,4.0,Sedán,Córdoba,9,11105.555556,2.5
1,Ford,Ranger,2012,2.3 Cd Xl Plus 4x2,140000,320000,2.0,Pick-Up,Entre Ríos,9,15555.555556,2.3
2,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,5.0,Hatchback,Bs.as. G.b.a. Sur,10,13200.000000,1.6
3,Ford,Ranger,2017,3.2 Cd Xls Tdci 200cv Automática,13000,798000,4.0,Pick-Up,Neuquén,4,3250.000000,3.2
4,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,3.0,Hatchback,Córdoba,8,13375.000000,1.4


In [ ]:
cars.Horsepower.value_counts().plot.bar()

# KPI

### What is KPI?

KPI (Key Performance Indicator) is a type of performance measurement. KPIs evaluate the success of an organization or of a particular activity (such as projects, programs, products and other initiatives) in which it engages. [[Wikipedia](https://en.wikipedia.org/wiki/Performance_indicator)]

Depending on the project you are working with it could be:
- number of new customers per month
- sum of the orders value per day
- Net Promoter Score (NPS)

And many many more...

### Question:

Let's imagine that our dataset has an additional column `Sold_date` (we will add some random data here on our own).

1. How many cars per month are sold?

Let's create `Sold_date` column! We already know how to add a new column to our dataframe. Here, we will assign each row a random date from 2019. To imitate real case scenario, where not all of the products are sold, we will leave some of the dates as NULL values.

In [ ]:
from random import randrange
import datetime 
import numpy as np

start_date = datetime.datetime(2019, 1, 1, 12, 0)

def create_new_date(start=start_date):
    # Creates a random date within a year distance from `start` date.
    
    # 30% of cars are not sold - they won't have a sold date.
    if randrange(10) > 2:
        # Generate random date from 2019.
        return start_date + datetime.timedelta(days=randrange(365))
    return np.nan

# Add a new column `Sold_date` to our dataframe.
cars['Sold_date'] = cars.Year.apply(lambda x: create_new_date())

In [ ]:
cars.head()

,Make,Model,Year,Variant,Kms,Price,Doors,Kind,Location,Age,KM_per_year,Horsepower,Sold_date
0,Volkswagen,Vento,2012,2.5 Luxury 170cv,99950,360000,4.0,Sedán,Córdoba,9,11105.555556,2.5,2019-08-18 12:00:00
1,Ford,Ranger,2012,2.3 Cd Xl Plus 4x2,140000,320000,2.0,Pick-Up,Entre Ríos,9,15555.555556,2.3,2019-03-22 12:00:00
2,Volkswagen,Fox,2011,1.6 Trendline,132000,209980,5.0,Hatchback,Bs.as. G.b.a. Sur,10,13200.000000,1.6,2019-05-11 12:00:00
3,Ford,Ranger,2017,3.2 Cd Xls Tdci 200cv Automática,13000,798000,4.0,Pick-Up,Neuquén,4,3250.000000,3.2,2019-03-17 12:00:00
4,Volkswagen,Gol,2013,1.4 Power 83cv 3 p,107000,146000,3.0,Hatchback,Córdoba,8,13375.000000,1.4,2019-07-13 12:00:00


In [ ]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Make         10000 non-null  object        
 1   Model        10000 non-null  object        
 2   Year         10000 non-null  int64         
 3   Variant      10000 non-null  object        
 4   Kms          10000 non-null  int64         
 5   Price        10000 non-null  int64         
 6   Doors        10000 non-null  float64       
 7   Kind         10000 non-null  object        
 8   Location     10000 non-null  object        
 9   Age          10000 non-null  int64         
 10  KM_per_year  10000 non-null  float64       
 11  Horsepower   10000 non-null  object        
 12  Sold_date    7071 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(4), object(6)
memory usage: 1015.8+ KB


## Saving the data to a file and reading it from a file  

Let's save this csv file and read that again from the file!

In [ ]:
cars.to_csv("cars.csv")

In [ ]:
cars_from_file = pd.read_csv("cars.csv")

In [ ]:
cars_from_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Make         10000 non-null  object 
 2   Model        10000 non-null  object 
 3   Year         10000 non-null  int64  
 4   Variant      10000 non-null  object 
 5   Kms          10000 non-null  int64  
 6   Price        10000 non-null  int64  
 7   Doors        10000 non-null  float64
 8   Kind         10000 non-null  object 
 9   Location     10000 non-null  object 
 10  Age          10000 non-null  int64  
 11  KM_per_year  10000 non-null  float64
 12  Horsepower   10000 non-null  float64
 13  Sold_date    7071 non-null   object 
dtypes: float64(3), int64(5), object(6)
memory usage: 1.1+ MB


## Datetime columns

Notice that datetime values when saved to file and read again, it changes to object data type. But we can transform it to datetime again very easily!

In [ ]:
cars_from_file['Sold_date'] = pd.to_datetime(cars_from_file['Sold_date'])

In [ ]:
cars_from_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   10000 non-null  int64         
 1   Make         10000 non-null  object        
 2   Model        10000 non-null  object        
 3   Year         10000 non-null  int64         
 4   Variant      10000 non-null  object        
 5   Kms          10000 non-null  int64         
 6   Price        10000 non-null  int64         
 7   Doors        10000 non-null  float64       
 8   Kind         10000 non-null  object        
 9   Location     10000 non-null  object        
 10  Age          10000 non-null  int64         
 11  KM_per_year  10000 non-null  float64       
 12  Horsepower   10000 non-null  float64       
 13  Sold_date    7071 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(5), object(5)
memory usage: 1.1+ MB


We will group our dataset by month (since we have the data only from one year) and calculate how many of them we have for each group:

In [ ]:
grouped = cars_from_file.groupby(by=cars_from_file.Sold_date.dt.month)[['Year']].size()
grouped.index.rename('Sold_date_month', inplace=True)
grouped

Sold_date_month
1.0     613
2.0     553
3.0     561
4.0     607
5.0     586
6.0     565
7.0     625
8.0     628
9.0     542
10.0    583
11.0    595
12.0    613
dtype: int64

In [ ]:
grouped.plot.bar()

Voila! Now, we can communicate how number of sold cars was changing in time :)

Extra materials:
- [Difference between loc and iloc functions](https://www.analyticsvidhya.com/blog/2020/02/loc-iloc-pandas/)
- [Apply, Map and ApplyMap](https://towardsdatascience.com/introduction-to-pandas-apply-applymap-and-map-5d3e044e93ff)
- [Transform function](https://towardsdatascience.com/when-to-use-pandas-transform-function-df8861aa0dcf)
- [Numpy library documentation](https://numpy.org/)
